### The code and data are adapted from:  https://medium.com/@vitalshchutski/french-nlp-entamez-le-camembert-avec-les-librairies-fast-bert-et-transformers-14e65f84c148

In [1]:
# !conda install torch
# !pip install fast-bert==1.9.1
# !mkdir model
# !mkdir finetuned_model

In [1]:
import torch
from fast_bert.data_cls import BertDataBunch 
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from fast_bert.metrics import fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor
from pathlib import Path
import pandas as pd
import logging



device_cuda = torch.device("cuda")
logging.basicConfig(
    handlers=[logging.FileHandler('logfile.log', 'w', 'utf-8')],
    format='%(levelname)s: %(message)s',
    datefmt='%m-%d %H:%M',
    level=logging.INFO #CRITICAL ERROR WARNING  INFO    DEBUG    NOTSET 
)
logger = logging.getLogger()

In [4]:
DATA_PATH = Path('./data/')
LOG_PATH = Path('./logs/')
MODEL_PATH = Path('./model_bert_multiligual/')
LABEL_PATH = Path('./labels/')

In [5]:
df = pd.read_csv('./data/bsv_chunk256_ba_mld_1201-1500.csv')

In [4]:
val_set = df.sample(frac=0.2, replace=False, random_state=42)
train_set = df.drop(index = val_set.index)
print('Nombre de commentaires dans le val_set:',len(val_set))
print('Nombre de commentaires dans le train_set:', len(train_set))
val_set.to_csv('./data/val_set.csv')
train_set.to_csv('./data/train_set.csv')

Nombre de commentaires dans le val_set: 860
Nombre de commentaires dans le train_set: 3441


In [5]:
labels = df.columns[1:3].to_list() 
with open('./labels/labels.txt', 'w') as f:
    for i in labels:
        f.write(i + "\n")

In [6]:
df_texts = pd.read_csv('./data/bsv_chunk256_raw_1001-1200.csv')
all_texts = df_texts['report_text'].to_list()
print('Nombre de bloc de texte:', len(all_texts))

Nombre de bloc de texte: 2801


### Création de LMDataBunch

In [7]:
databunch_lm = BertLMDataBunch.from_raw_corpus(
                    data_dir=DATA_PATH,
                    text_list=all_texts,
                    tokenizer='bert-base-multilingual-cased',
                    batch_size_per_gpu=4, #was 16, even 8 won't do
                    max_seq_length=256, #was 512
                    multi_gpu=False,
                    model_type='bert',
                    logger=logger)

### Création de LMLearner

In [8]:
lm_learner = BertLMLearner.from_pretrained_model(
                            dataBunch=databunch_lm,
                            pretrained_path='bert-base-multilingual-cased',
                            output_dir=MODEL_PATH,
                            metrics=[],
                            device=device_cuda,
                            logger=logger,
                            multi_gpu=False,
                            logging_steps=50,
                            is_fp16=True) #was true with gpu

In [9]:
lm_learner.fit(epochs=2, #was 30
            lr=1e-4,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


C:\Users\Raja\Documents\SJI\tools\anaconda3\envs\fast-camembert\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


C:\Users\Raja\Documents\SJI\tools\anaconda3\envs\fast-camembert\lib\site-packages\torch\optim\lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0


(542, 1.7026500087162666)

In [10]:
lm_learner.validate()

{'loss': 0.0999575922531741, 'perplexity': 1.1051239967346191}

In [11]:
lm_learner.save_model()

In [12]:
del lm_learner

### Création de databunch pour la classification

In [13]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='bert-base-multilingual-cased',
                          train_file='train_set.csv',
                          val_file='val_set.csv',
                          label_file='labels.txt',
                          text_col='report_text',
                          label_col=['Bioagressor','Disease'],
                          batch_size_per_gpu=8,
                          max_seq_length=256,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='bert')

### Création de Learner

In [14]:
metrics = [{'name': 'fbeta', 'function': fbeta}, {'name': 'roc_auc', 'function': roc_auc}]
OUTPUT_DIR = Path('./finetuned_model_bert_multiligual')
WGTS_PATH = Path('model_bert_multiligual/model_out/pytorch_model.bin')

In [15]:
# issue fast-bert pos_weight <= downgrade to 1.9.1 solve the prob
cl_learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path='model_bert_multiligual/model_out',
                        metrics=metrics,
                        device=device_cuda, #was device_cuda
                        logger=logger,
                        output_dir=OUTPUT_DIR,
                        finetuned_wgts_path=WGTS_PATH,
                        warmup_steps=300,
                        multi_gpu=False,
                        multi_label=True,
                        is_fp16=True,#True when is cuda
                        logging_steps=50)

In [16]:
cl_learner.fit(epochs=5,# was 30
            lr=2e-5,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0


(2155, 0.2776137579206524)

In [18]:
cl_learner.validate()

{'loss': 0.4490230752638093,
 'fbeta': 0.3795219957828522,
 'roc_auc': 0.9105695789865872}

In [19]:
cl_learner.save_model()

### Prédictions

In [20]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model_bert_multiligual/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='bert',
                do_lower_case=False)

In [25]:
#cas disease: 0, bioagressor: 1 - cicadelle
predictor.predict("Cicadelles La cicadelle Edwardsiana est toujours observée sur les parcelles en été")

C:\Users\Raja\Documents\SJI\tools\anaconda3\envs\fast-camembert\lib\site-packages\pandas\core\frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


[('bioagressor', 0.99853515625), ('disease', 0.0018606185913085938)]

In [26]:
predictor.predict("election américane")

[('bioagressor', 0.0009365081787109375), ('disease', 0.0008039474487304688)]

In [27]:
#cas disease: 0, bioagressor: 0 - texte sur mouche de carotte mais pas de rique
predictor.predict("mouche de la carotte :ajouter trichloronate")

[('bioagressor', 0.966796875), ('disease', 0.0004973411560058594)]

In [28]:
#cas disease: 0, bioagressor: 0 - oïdium : pas d'intervention 
predictor.predict("oïdium : pas d'intervention dans l'immediat les conditions très chaudes et l'absence de rosées nocturnes sont défavorables à cette maladie. aucun symptôme n'a encore été observé.")

[('disease', 0.99755859375), ('bioagressor', 0.001789093017578125)]

In [29]:
#cas disease: 1, bioagressor: 0 - oidium  => good
predictor.predict("oïdium du pommierce champignon est à l'heure actuelle en pleine fructification. il est nécessaire d'ajouter un antioïdium aux bouillies pour assurer'la protection du jeune feuillage.")

[('disease', 0.99853515625), ('bioagressor', 0.0029697418212890625)]

In [30]:
#cas disease: 1, bioagressor: 0 - mildiou
predictor.predict("vigilant : en particulier vis-à-vis du mildiou, de l’oïdium et de la bactériose / cladosporiose. Les prévisions pour les prochains jours restent peu favorables à l’expression de la bactériose et la cladosporiose, si elles se confirment. Mildiou (Pseudoperonospora cubensis) : Le modèle annonce un risque élevé pour toutes les dates de plantation avec les données de la station de Thurageau. Avec les données de la station de Maulay, les plantations en S25 et S26 montrent un risque modéré. Le risque est un peu plus élevé dans le sud de la Charente- Maritime que dans le Poitou. Niveau de risque Faible Moyen Élevé Très élevé Indice : Log (Nb de taches/unité de surface) -14 à -9 -9 à -4 -4 +4 Équivalent en unité de surface 1 tâche par hectare par 100 m2 1 tâche par 100 m2 par m2 1 tâche par m2à 1 % de surface atteinte 1 % à 100 % de surface atteinte Évaluation du risque : les conditions restent favorables à ce microorganisme (qui n’est pas un champignon, mais proche d’une algue). Les BSV sont disponibles en accès direct sur le site  (rubrique : Nos publications - Bulletin de santé du végétal) ou par abonnement en ligne gratuit sur le site  BSV CULTURES LÉGUMIÈRES DE PLEIN")

[('disease', 0.99853515625), ('bioagressor', 0.004100799560546875)]

In [31]:
#cas disease: 0, bioagressor: 0 - texte sur trump trump
predictor.predict("L'avance de Donald Trump dans cet Etat où 4,9 millions d'électeurs ont voté, a fondu vendredi 6 novembre. Le candidat républicain compte")

[('disease', 0.0012159347534179688), ('bioagressor', 0.0006985664367675781)]

In [32]:
#cas disease: 1, bioagressor: 1 - #mildiou (removed) mais pas de risque
predictor.predict("Pomme de te r re du 17 mai 2013 BS omme Ce - N° 9 N° 9 En résumé : Risque  au 17 mai : entrée en phase de risque pour les variétés sensibles pour 5 secteurs. Pour autant, le seuil de nuisibilité n’est pas atteint. Utilisation du modèle Mileos® (www.mileos.fr) Le BSV pomme de terre de la région Centre mobilise le modèle Mileos® qui se base sur le cycle épidémique de Phytophtora infestans. • Quand démarre le risque  ? Le suivi du nombre des générations de  est un bon indicateur pour connaître le début de la période à risque de cette maladie. En fonction de la sensibilité variétale, le risque démarre : - à la sortie de")

[('disease', 0.0010042190551757812), ('bioagressor', 0.0007381439208984375)]

In [34]:
#cas disease: 0, bioagressor: 0 - zero mot clé
predictor.predict("par jour. Sur les autres secteurs, en production de carotte, les captures sont nulles. Évaluation du risque")

[('bioagressor', 0.000904083251953125), ('disease', 0.00080108642578125)]

In [36]:
#cas disease: 0, bioagressor: 0 + mildiou
predictor.predict("7 Action pilotée par le ministère chargé de l’agriculture mildiou, avec l’appui financier de l’Office National de l’Eau et des Milieux Aquatiques (ONEMA), par les crédits issus de la redevance pour pollutions diffuses attribués au")

[('disease', 0.9560546875), ('bioagressor', 0.00047278404235839844)]

In [37]:
#cas disease: 0, bioagressor: 0
predictor.predict("financement du plan Ecophyto. Ce bulletin est rédigé par l'ACPEL avec la collaboration de référents par culture (techniciens des Chambres d'Agriculture de la Charente, de la Charente-Maritime et d'Indre et Loire et de la Vienne) sur la base d'observations réalisées par des producteurs et techniciens : Charentes- Alliance, les entreprises de production de melon, la coopérative AGROLEG, la coopérative UNIRE, des producteurs d'Agrobio Poitou-Charentes. Ce bulletin est réalisé à partir d'observations ponctuelles. Il a pour vocation de donner une tendance de la situation sanitaire régionale. Celle-ci ne peut être transposée telle quelle dans les parcelles de production légumières (conditions très variables). La Chambre Régionale d'Agriculture de Poitou-Charentes et le rédacteur dégagent toute responsabilité quant aux décisions prises par les producteurs pour la protection de leurs cultures. Elle les invite à prendre ces décisions sur la base des observation s qu'ils auront réalisées dans leurs parcelles. Les")

[('bioagressor', 0.0015430450439453125), ('disease', 0.00046563148498535156)]

In [38]:
#cas disease: 0, bioagressor: 0 - french crop usasge
predictor.predict("Ensemble de plantes cultivées pour leurs fruits ou leurs graines riches en matières grasses (lipides). De ces fruits et graines sont extrait une huile à usage alimentaire humaine, alimentaire animal ou industriel. Les résidus de l'extraction constituent les tourteaux utilisés pour l'alimentation animale.")

[('bioagressor', 0.0010480880737304688), ('disease', 0.0006718635559082031)]

In [39]:
#cas disease: 0, bioagressor: 0 - french crop usasge
predictor.predict("Orge semé après le 1er février, principalement de mars à mai. Orge de printemps est toujours à deux rangs")

[('bioagressor', 0.0011920928955078125), ('disease', 0.0006093978881835938)]

In [40]:
#cas disease: 1, bioagressor: 0 - tweet - jaunisse
predictor.predict("des parcelles qui ont eu une croissance presque nulle depuis cet été en majeur partie dû à la jaunisse.")

[('disease', 0.0008625984191894531), ('bioagressor', 0.0008459091186523438)]

In [41]:
#cas disease: 1, bioagressor: 0 - tweet - sécheresse
predictor.predict("Parfois,on nous demande. Que faites-vous,par cette sécheresse?")

[('disease', 0.001220703125), ('bioagressor', 0.0007977485656738281)]

In [42]:
#cas disease: 0, bioagressor: 1 - tweet - pucerons

predictor.predict("Attention pucerons dans les blés")

[('bioagressor', 0.99853515625), ('disease', 0.002124786376953125)]

### Analyse the fine-tuned model

In [2]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model_bert_multiligual/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='bert',
                do_lower_case=False)

In [3]:
#evaluation on validation set
df_val = pd.read_csv('./data/val_set.csv')

#predictor.get_learner()

In [4]:
df_val.Disease = df_val.Disease.astype(int)
df_val.tail()

,Unnamed: 0,report_text,Bioagressor,Disease,hazard_list,source_name
855,2112,MILDIOU DU TOURNESOL (Plasmopara halstedii) No...,0,0,NaN,Note_Commune-Mildiou-Tournesol-2012_cle0b4a54
856,1146,. . . . . . . . . . . . . . . . . . . . . . . ...,0,0,NaN,draaf.centre-val-de-loire.agriculture.gouv.fr_...
857,3528,"conditions automnales et hivernales, particuli...",0,0,NaN,www.paysdelaloire.chambagri.fr_fileadmin_docum...
858,4062,teigne des crucifères (Plutella xylostella) à ...,1,0,#teigne des crucifères #mineuse,BSV_-_Legumes_no10-1_cle0f73f2
859,4246,– N ° rédigé par Hervé FRANCOIS - Chambre d’a...,0,1,#septoriose,20130430_bsv_grandescultures_13_cle4d3d84


In [5]:
batch_predictions = predictor.predict_batch(df_val.report_text.to_list())

C:\Users\Raja\Documents\SJI\tools\anaconda3\envs\fast-camembert\lib\site-packages\pandas\core\frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [6]:
batch_predictions[:10]

[[('bioagressor', 0.008679624646902084), ('disease', 0.003354396903887391)],
 [('bioagressor', 0.008183608762919903), ('disease', 0.0036047056782990694)],
 [('disease', 0.9877176284790039), ('bioagressor', 0.9783329367637634)],
 [('disease', 0.963326096534729), ('bioagressor', 0.952403724193573)],
 [('bioagressor', 0.9215682148933411), ('disease', 0.8805903792381287)],
 [('bioagressor', 0.006274393294006586), ('disease', 0.006084537133574486)],
 [('bioagressor', 0.985973596572876), ('disease', 0.022453520447015762)],
 [('bioagressor', 0.01663808710873127), ('disease', 0.008762485347688198)],
 [('bioagressor', 0.8687992095947266), ('disease', 0.5848673582077026)],
 [('bioagressor', 0.008594009093940258), ('disease', 0.0032442237716168165)]]

In [7]:
#dict(batch_predictions[1]).values()
list_y_pred = [ dict(pred) for pred in batch_predictions]
#list_y_pred[-5:]

In [8]:
df_y_pred = pd.DataFrame(list_y_pred, columns =['bioagressor', 'disease']) 
df_y_pred = df_y_pred.rename(columns={"bioagressor": "Bioagressor", "disease": "Disease"})
df_y_pred.tail()

,Bioagressor,Disease
855,0.038225,0.984230
856,0.011789,0.002736
857,0.007280,0.004219
858,0.988084,0.041641
859,0.045507,0.979611


In [9]:
df_y_pred.describe()

,Bioagressor,Disease
count,860.000000,860.000000
mean,0.406573,0.255234
std,0.449311,0.403972
min,0.004701,0.001123
25%,0.009296,0.003802
50%,0.045812,0.008797
75%,0.963721,0.523773
max,0.993775,0.996270


In [10]:
df_y_real = pd.DataFrame(df_val, columns=['Bioagressor', 'Disease'])
df_y_real.tail()

,Bioagressor,Disease
855,0,0
856,0,0
857,0,0
858,1,0
859,0,1


In [11]:
df_y_real.describe()

,Bioagressor,Disease
count,860.000000,860.000000
mean,0.366279,0.201163
std,0.482067,0.401103
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,1.000000,0.000000
max,1.000000,1.000000


In [12]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

In [13]:
precision_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None) #If None, the scores for each class are returned

array([0.78409091, 0.69585253])

In [14]:
precision_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5, average='weighted')

0.7528096820551209

In [15]:
recall_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None)

array([0.87619048, 0.87283237])

In [16]:
recall_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average='weighted')

0.875

In [17]:
f1_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None)

array([0.82758621, 0.77435897])

In [18]:
f1_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average='weighted')

0.8087167166731892

In [19]:
accuracy_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5)

0.7941860465116279

In [20]:
accuracy_score(y_true=df_y_real.Bioagressor.values, y_pred=df_y_pred.Bioagressor.values > 0.5)


0.8662790697674418

In [21]:
accuracy_score(y_true=df_y_real.Disease.values, y_pred=df_y_pred.Disease.values > 0.5 )


0.8976744186046511

In [22]:
from sklearn.metrics import average_precision_score
average_precision_score(df_y_real.values, df_y_pred.values, average=None)

array([0.85153045, 0.70837686])

In [23]:
from sklearn.metrics import average_precision_score
average_precision_score(df_y_real.values, df_y_pred.values, average='weighted')

0.8007813268823349

In [24]:
df_results = pd.concat([df_val, df_y_pred > 0.5], axis=1, ignore_index=True)

In [43]:
df_results.to_csv('predictions_bert_multilingual.csv')